In [ ]:
#imports 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree  
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, mean_absolute_error, accuracy_score, precision_score, recall_score
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
!pip install dtreeviz

Loading the data

In [ ]:
df = pd.read_csv('/content/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df

In [ ]:
df.info()

###convertendo a coluna 19 (TotalCharges) para float e Verificando se têm algum valor nulo.

Nesta coluna existem alguns valores que foram declarados como vazio (" "). Olhando atentamente para os dado em comun entre as linhas com este valor, nota-se que todos possuem um Tenure igual a 0. Ou seja, estes são clientes que acabaram de assinar o contrato e ainda não receberam nenhuma cobrança. 

Por este motivo eu coloquei 0 no totalCharges abaixo.

In [ ]:
df['TotalCharges'][df['TotalCharges']==" "] = 0

In [ ]:
df['TotalCharges']=pd.to_numeric(df["TotalCharges"], downcast="float")

In [ ]:
data = df.copy()#copiando para utilizar nos gráficos lá no final

In [ ]:
df.info()

In [ ]:
df[df.isna().any(axis=1)]

In [ ]:
df[df.isnull().any(axis=1)]

**Nenhum valor é nulo.**

###Substituindo os dados categóricos por numéricos.

In [ ]:
#separando as colunas que precisam ser alteradas
colunas =['gender',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'Churn']

In [ ]:
labels = {}#este será o dicionário que ficará salvo qual é a conversão

In [ ]:
for x in colunas:
  df[x],labels[x] = df[x].factorize()

In [ ]:
labels

In [ ]:
df

###Dividindo em Features e Target.

In [ ]:

features = df.drop(["Churn", "customerID"], axis=1)
features

In [ ]:
target= df['Churn']
target

###Plotando a Arvore

In [ ]:

decisionTree = DecisionTreeClassifier(max_depth=4)

In [ ]:
# Treinar o modelo
model = decisionTree.fit(features,target)

In [ ]:
features.columns

In [ ]:
# Gera a figura
fig = plt.figure(figsize=(30,25))

# Gera a árvore
_ = tree.plot_tree(model,
               feature_names = features.columns,
               class_names = target.apply(str).unique(),
               filled=True)
fig.savefig("decistion_tree.png")

In [ ]:
from dtreeviz.trees import dtreeviz # remember to load the package

viz = dtreeviz(model, features, target,
                feature_names = features.columns,
                class_names = list(target.apply(str).unique())
                )

viz

In [ ]:
viz.save("ArvoreChurn.svg")

###Segmentação dos dados e Validação do modelo

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(features,target, test_size=0.2, random_state=1)

In [ ]:
# Treinar o modelo
model = decisionTree.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, mean_absolute_error, accuracy_score, precision_score, recall_score, f1_score
accuracy_score(y_test,y_pred)

In [ ]:
precision_score(y_test,y_pred)

In [ ]:
recall_score(y_test,y_pred)

In [ ]:
f1_score(y_test,y_pred)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
mat =confusion_matrix(y_test, y_pred)
axes = sns.heatmap(mat, square=True, annot=True, fmt='d', cbar=True)

###importância de cada feature

In [ ]:
d = {i:j for i,j in zip(features.columns, model.feature_importances_)}

In [ ]:
sorted(d.items(), key=lambda item: item[1], reverse=True)

###Visualização com Gráficos.

In [ ]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data.head(5)

In [ ]:
c_labels = ['Não', 'Sim']
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=c_labels, values=df['Churn'].value_counts(), name="Churn"))
fig.update_traces(hole=.4, hoverinfo="label+percent+name", textfont_size=16)
fig.update_layout(
    title_text="Porcentagem de pessoas que cometeram o churn",
    annotations=[dict(text='Churn', font_size=20, showarrow=False)])
fig.show()

In [ ]:
fig = px.histogram(data, x="Churn", color="Contract", barmode="group", title="<b>Distribuição do Churn por tipo de contrato<b>")
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

In [ ]:
fig = px.pie(data, values=data.Contract.value_counts(), names= data.Contract.value_counts().index, title='Distribuição de planos', width=440, height=480)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

------------------------------------------------------------------------------
- **Está claro que quanto mais longo o plano menor é o Churn. 88% das pessoas que saíram possuíam um plano mensal.**

----------------------------------------------------------------------------

In [ ]:
sairam = {'Mensal':1655, '1-Ano':166, "2-Anos": 48}
naoSairam = {'Mensal':2220, '1-Ano':1307, "2-Anos": 1647}
total= {'Mensal':3875, '1-Ano':1473, "2-Anos": 1695}
qtdClientes = data.shape[0]

In [ ]:
print(f" porcentagem dos que cometeram o churn e tinha um plano mensal {sairam['Mensal']/data['Churn'].value_counts()['Yes']} ")

In [ ]:
color_map = {"Yes": "#FF97FF", "No": "#AB63FA"}
fig = px.histogram(data, x="Churn", color="OnlineSecurity", barmode="group", title="<b>Distribuição do Churn para Online Security</b>", color_discrete_map=color_map)
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

- **É NOTÁVEL A QUANTIDADE DE *CHURN* QUANDO NÃO TEM OnlineSecurity.**



In [ ]:
fig = px.box(data, x='Churn', y = 'tenure')

# Update yaxis properties
fig.update_yaxes(title_text='Tenure (Months)', row=1, col=1)
# Update xaxis properties
fig.update_xaxes(title_text='Churn', row=1, col=1)

# Update size and title
fig.update_layout(autosize=True, width=700, height=550,
    title_font=dict(size=25, family='Courier'),
    title='<b>Tenure vs Churn</b>',
)

fig.show()

- **O gráfico acima mostra que a maior parte das pessoas que saem contrataram o serviço há pouco tempo. Ou seja, mais um indício que os planos mais longos (1 ou 2 anos) são mais vantajosos.**

In [ ]:
fig = px.histogram(data, x="Churn", color="InternetService", barmode="group", title="<b>Distribuição do Churn para os serviços de internet</b>")
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

In [ ]:
fig = px.pie(data, values=data.InternetService.value_counts(), names= data.InternetService.value_counts().index, title='Distribuição de Serviços de internet', width=440, height=480)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
data[data["Churn"]=='Yes']['InternetService'].value_counts()['Fiber optic'] / sum(data[data["Churn"]=='Yes']['InternetService'].value_counts())

- **A internet por fibra ótica parece ter algum problema, visto que eles representam 44% dos contratos e 69% do Churn. Em outras palavras, 69% das pessoas que saíram possuíam internet por fibra ótica.**

CARACTERÍSTICAS RECORRENTES ENTRE AS PESSOAS QUE CANCELAM A ASSINATURA (CHURN):

1. - Contrataram o plano mensal
2. - Não possuem segurança online no plano (online security)
3. - São clientes a pouco tempo.
4. - Possuem internet por fibra ótica.